# BEE 4750 Homework 4: Linear Programming and Capacity Expansion

**Name**: Kenneth Wu, Arron Chang

**ID**: kcw53, ac2787

> **Due Date**
>
> Thursday, 11/07/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to formulate and solve a resource allocation
    problem using linear programming.
-   Problem 2 asks you to formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   Problem 3 (5750 only) asks you to add a CO<sub>2</sub> constraint to
    the capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/code/hw4-job-hunt`
   Installed SentinelArrays ─────── v1.4.5
   Installed InlineStrings ──────── v1.4.2
   Installed BenchmarkTools ─────── v1.5.0
   Installed Pango_jll ──────────── v1.52.2+0
   Installed HiGHS_jll ──────────── v1.7.2+0
   Installed CodecBzip2 ─────────── v0.8.3
   Installed MutableArithmetics ─── v1.4.6
   Installed ArgCheck ───────────── v2.3.0
   Installed CommonSubexpressions ─ v0.3.0
   Installed DiffRules ──────────── v1.15.1
   Installed DisplayAs ──────────── v0.1.6
   Installed TranscodingStreams ─── v0.11.1
   Installed DiffResults ────────── v1.1.0
   Installed HiGHS ──────────────── v1.9.2
   Installed JuMP ───────────────── v1.22.2
   Installed MarkdownTables ─────── v1.1.0
   Installed ForwardDiff ────────── v0.10.36
   Installed MathOptInterface ───── v1.31.0
Precompiling project...
  ✓ ArgCheck
  ✓ InlineStrings
  ✓ DisplayAs
  ✓ TranscodingStreams
  ✓ CommonSubexpressions
  ✓ FilePathsBase
  ✓ PooledArrays
  ✓ StringManipul

In [2]:
using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables

[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


## Problems (Total: 50/60 Points)

### Problem 1 (20 points)

A farmer has access to a pesticide which can be used on corn, soybeans,
and wheat fields, and costs \$70/ha-yr to apply. The crop yields the
farmer can obtain following crop yields by applying varying rates of
pesticides to the field are shown in
<a href="#tbl-yields" class="quarto-xref">Table 1</a>.

| Application Rate (kg/ha) | Soybean (kg/ha) | Wheat (kg/ha) | Corn (kg/ha) |
|:------------------------:|:---------------:|:-------------:|:------------:|
|            0             |      2900       |     3500      |     5900     |
|            1             |      3800       |     4100      |     6700     |
|            2             |      4400       |     4200      |     7900     |

Table 1: Crop yields from applying varying pesticide rates for Problem
1.

The costs of production, *excluding pesticides*, for each crop, and
selling prices, are shown in
<a href="#tbl-costs" class="quarto-xref">Table 2</a>.

|   Crop   | Production Cost (\$/ha-yr) | Selling Price (\$/kg) |
|:--------:|:--------------------------:|:---------------------:|
| Soybeans |            350             |         0.36          |
|  Wheat   |            280             |         0.27          |
|   Corn   |            390             |         0.22          |

Table 2: Costs of crop production, excluding pesticides, and selling
prices for each crop.

Recently, environmental authorities have declared that farms cannot have
an *average* application rate on soybeans, wheat, and corn which exceeds
0.8, 0.7, and 0.6 kg/ha, respectively. The farmer has asked you for
advice on how they should plant crops and apply pesticides to maximize
profits over 130 total ha while remaining in regulatory compliance if
demand for each crop (which is the maximum the market would buy) this
year is 250,000 kg?

**In this problem**:

-   Formulate a linear program for this resource allocation problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations). **Tip: Make sure that all of your constraints are
    linear**.
-   Implement the program in `JuMP.jl` and find the solution. How many
    ha should the farmer dedicate to each crop and with what pesticide
    application rate(s)? How much profit will the farmer expect to make?
-   The farmer has an opportunity to buy an extra 10 ha of land. How
    much extra profit would this land be worth to the farmer? Discuss
    why this value makes sense and whether you would recommend the
    farmer should make the purchase.

In [33]:
m = Model(HiGHS.Optimizer)

soybean_price = 0.36
wheat_price = 0.27
corn_price = 0.22

soybean_cost = 350
wheat_cost = 280
corn_cost = 390

pest_cost = 70

soybean_yield0 = 2900
soybean_yield1 = 3800
soybean_yield2 = 4400
wheat_yield0 = 3500
wheat_yield1 = 4100
wheat_yield2 = 4200
corn_yield0 = 5900
corn_yield1 = 6700
corn_yield2 = 7900

@variable(m, s0 >= 0)  # soybean hectares, non-negative, app rate 0
@variable(m, s1 >= 0)  # soybean hectares, app rate 1
@variable(m, s2 >= 0)  # soybean hectares, app rate 2
@variable(m, w0 >= 0)  # wheat hectares, non-negative, app rate 0
@variable(m, w1 >= 0)  # wheat hectares, app rate 1
@variable(m, w2 >= 0)  # wheat hectares, app rate 2
@variable(m, c0 >= 0)  # corn hectares, non-negative, app rate 0
@variable(m, c1 >= 0)  # corn hectares, app rate 1
@variable(m, c2 >= 0)  # corn hectares, app rate 2


@objective(m, Max, 
    ((soybean_price*soybean_yield0-soybean_cost-pest_cost*0) * s0) +
    ((soybean_price*soybean_yield1-soybean_cost-pest_cost*1) * s1) +
    ((soybean_price*soybean_yield2-soybean_cost-pest_cost*2) * s2) +
    ((wheat_price*wheat_yield0-wheat_cost-pest_cost*0) * w0) +
    ((wheat_price*wheat_yield1-wheat_cost-pest_cost*1) * w1) +
    ((wheat_price*wheat_yield2-wheat_cost-pest_cost*2) * w2) +
    ((corn_price*corn_yield0-corn_cost-pest_cost*0) * c0) +
    ((corn_price*corn_yield1-corn_cost-pest_cost*1) * c1) +
    ((corn_price*corn_yield2-corn_cost-pest_cost*2) * c2)
    )

@constraint(m, land, s0+s1+s2+w0+w1+w2+c0+c1+c2 <= 130)

@constraint(m, s1 + 2 * s2 - 0.8 * (s0 + s1 + s2) <= 0)
@constraint(m, w1 + 2 * w2 - 0.7 * (w0 + w1 + w2) <= 0)
@constraint(m, c1 + 2 * c2 - 0.6 * (c0 + c1 + c2) <= 0)

@constraint(m, 2900*s0 + 3800*s1 + 4400*s2 <= 250000)
@constraint(m, 3500*w0 + 4100*w1 + 4200*w2 <= 250000)
@constraint(m, 5900*c0 + 6700*c1 + 7900*c2 <= 250000)


optimize!(m)

println("Optimal Soybean areas:")
println("Rate 0 (0 kg/ha): ", value(s0), " ha")
println("Rate 1 (1 kg/ha): ", value(s1), " ha")
println("Rate 2 (2 kg/ha): ", value(s2), " ha")

println("\nOptimal Wheat areas:")
println("Rate 0 (0 kg/ha): ", value(w0), " ha")
println("Rate 1 (1 kg/ha): ", value(w1), " ha")
println("Rate 2 (2 kg/ha): ", value(w2), " ha")

println("\nOptimal Corn areas:")
println("Rate 0 (0 kg/ha): ", value(c0), " ha")
println("Rate 1 (1 kg/ha): ", value(c1), " ha")
println("Rate 2 (2 kg/ha): ", value(c2), " ha")

# Calculate total pesticide application rates based on allocated hectares
total_pesticide_soybeans = value(s1) * 1 + value(s2) * 2
total_pesticide_wheat = value(w1) * 1 + value(w2) * 2
total_pesticide_corn = value(c1) * 1 + value(c2) * 2

println("\nTotal Pesticide Application Rates (kg/ha):")
println("Soybeans: ", total_pesticide_soybeans, " kg/ha")
println("Wheat: ", total_pesticide_wheat, " kg/ha")
println("Corn: ", total_pesticide_corn, " kg/ha")

println("\nTotal Profit: ", objective_value(m), " dollars")

@show shadow_price(land)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 8e+03]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+05]
Presolving model
7 rows, 9 cols, 27 nonzeros  0s
7 rows, 9 cols, 27 nonzeros  0s
Presolve : Reductions: rows 7(-0); columns 9(-0); elements 27(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.8170297740e+02 Ph1: 7(28.3601); Du: 9(181.703) 0s
          7     1.1674116702e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  1.1674116702e+05
HiGHS run time      :          0.00
Optimal Soybean areas:
Rate 0 (0 kg/ha): 13.812154696132593 ha
Rate 1 (1 kg/ha): 55.24861878453038 ha
Rate 2 (2 kg/ha): 0.0 ha

Optimal Wheat areas:
Rate 0 (0 kg/ha): 6.743306417339564 ha
Rate 1 (1 kg/ha): 15.73438164045898 ha
Rate 2 (2 kg/ha)

729.4000000000001

In [34]:
m = Model(HiGHS.Optimizer)

soybean_price = 0.36
wheat_price = 0.27
corn_price = 0.22

soybean_cost = 350
wheat_cost = 280
corn_cost = 390

pest_cost = 70

soybean_yield0 = 2900
soybean_yield1 = 3800
soybean_yield2 = 4400
wheat_yield0 = 3500
wheat_yield1 = 4100
wheat_yield2 = 4200
corn_yield0 = 5900
corn_yield1 = 6700
corn_yield2 = 7900

@variable(m, s0 >= 0)  # soybean hectares, non-negative, app rate 0
@variable(m, s1 >= 0)  # soybean hectares, app rate 1
@variable(m, s2 >= 0)  # soybean hectares, app rate 2
@variable(m, w0 >= 0)  # wheat hectares, non-negative, app rate 0
@variable(m, w1 >= 0)  # wheat hectares, app rate 1
@variable(m, w2 >= 0)  # wheat hectares, app rate 2
@variable(m, c0 >= 0)  # corn hectares, non-negative, app rate 0
@variable(m, c1 >= 0)  # corn hectares, app rate 1
@variable(m, c2 >= 0)  # corn hectares, app rate 2


@objective(m, Max, 
    ((soybean_price*soybean_yield0-soybean_cost-pest_cost*0) * s0) +
    ((soybean_price*soybean_yield1-soybean_cost-pest_cost*1) * s1) +
    ((soybean_price*soybean_yield2-soybean_cost-pest_cost*2) * s2) +
    ((wheat_price*wheat_yield0-wheat_cost-pest_cost*0) * w0) +
    ((wheat_price*wheat_yield1-wheat_cost-pest_cost*1) * w1) +
    ((wheat_price*wheat_yield2-wheat_cost-pest_cost*2) * w2) +
    ((corn_price*corn_yield0-corn_cost-pest_cost*0) * c0) +
    ((corn_price*corn_yield1-corn_cost-pest_cost*1) * c1) +
    ((corn_price*corn_yield2-corn_cost-pest_cost*2) * c2)
    )

@constraint(m, land, s0+s1+s2+w0+w1+w2+c0+c1+c2 <= 140)

@constraint(m, s1 + 2 * s2 - 0.8 * (s0 + s1 + s2) <= 0)
@constraint(m, w1 + 2 * w2 - 0.7 * (w0 + w1 + w2) <= 0)
@constraint(m, c1 + 2 * c2 - 0.6 * (c0 + c1 + c2) <= 0)

@constraint(m, 2900*s0 + 3800*s1 + 4400*s2 <= 250000)
@constraint(m, 3500*w0 + 4100*w1 + 4200*w2 <= 250000)
@constraint(m, 5900*c0 + 6700*c1 + 7900*c2 <= 250000)


optimize!(m)

println("\nTotal Profit: ", objective_value(m), " dollars")
diff = objective_value(m) - 116741.16702082448
println("\nExtra Profit: ", value(diff), " dollars")
@show shadow_price(land)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 8e+03]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+05]
Presolving model
7 rows, 9 cols, 27 nonzeros  0s
7 rows, 9 cols, 27 nonzeros  0s
Presolve : Reductions: rows 7(-0); columns 9(-0); elements 27(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.8170297740e+02 Ph1: 7(28.3601); Du: 9(181.703) 0s
          7     1.2403516702e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  1.2403516702e+05
HiGHS run time      :          0.00

Total Profit: 124035.16702082445 dollars

Extra Profit: 7293.999999999971 dollars
shadow_price(land) = 729.4000000000001


729.4000000000001

### Problem 2 (30 points)

For this problem, we will use hourly load (demand) data from 2013 in New
York’s Zone C (which includes Ithaca). The load data is loaded and
plotted below in <a href="#fig-demand" class="quarto-xref">Figure 1</a>.

In [1]:
# load the data, pull Zone C, and reformat the DataFrame
NY_demand = DataFrame(CSV.File("data/2013_hourly_load_NY.csv"))
rename!(NY_demand, :"Time Stamp" => :Date)
demand = NY_demand[:, [:Date, :C]]
rename!(demand, :C => :Demand)
demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(demand.Hour, demand.Demand, xlabel="Hour of Year", ylabel="Demand (MWh)", label=:false)

Next, we load the generator data, shown in
<a href="#tbl-generators" class="quarto-xref">Table 3</a>. This data
includes fixed costs (\$/MW installed), variable costs (\$/MWh
generated), and CO<sub>2</sub> emissions intensity (tCO<sub>2</sub>/MWh
generated).

In [1]:
gens = DataFrame(CSV.File("data/generators.csv"))

Finally, we load the hourly solar and wind capacity factors, which are
plotted in <a href="#fig-cf" class="quarto-xref">Figure 2</a>. These
tell us the fraction of installed capacity which is expected to be
available in a given hour for generation (typically based on the average
meteorology).

In [1]:
# load capacify factors into a DataFrame
cap_factor = DataFrame(CSV.File("data/wind_solar_capacity_factors.csv"))

# plot January capacity factors
p1 = plot(cap_factor.Wind[1:(24*31)], label="Wind")
plot!(cap_factor.Solar[1:(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

p2 = plot(cap_factor.Wind[4344:4344+(24*31)], label="Wind")
plot!(cap_factor.Solar[4344:4344+(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

display(p1)
display(p2)

You have been asked to develop a generating capacity expansion plan for
the utility in Riley County, NY, which currently has no existing
electrical generation infrastructure. The utility can build any of the
following plant types: geothermal, coal, natural gas combined cycle gas
turbine (CCGT), natural gas combustion turbine (CT), solar, and wind.

While coal, CCGT, and CT plants can generate at their full installed
capacity, geothermal plants operate at maximum 85% capacity, and solar
and wind available capacities vary by the hour depend on the expected
meteorology. The utility will also penalize any non-served demand at a
rate of \$10,000/MWh.

**In this problem**:

-   Formulate a linear program for this capacity expansion problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations).
-   Implement your linear program in `JuMP.jl`. Find the optimal
    solution. How much should the utility build of each type of
    generating plant? What will the total cost be? How much energy will
    be non-served?
-   What fraction of annual generation does each plant type produce? How
    does this compare to the breakdown of built capacity that you found
    in Problem 1.5? Do these results make sense given the generator
    data?
-   Make a plot of the electricity price in each hour. Discuss any
    trends that you see.

> **Significant Digits**
>
> Use `round(x; digits=n)` to report values to the appropriate
> precision! If your number is on a different order of magnitude and you
> want to round to a certain number of significant digits, you can use
> `round(x; sigdigits=n)`.

> **Getting Variable Output Values**
>
> `value.(x)` will report the values of a `JuMP` variable `x`, but it
> will return a special container which holds other information about
> `x` that is useful for `JuMP`. This means that you can’t use this
> output directly for further calculations. To just extract the values,
> use `value.(x).data`.

> **Suppressing Model Command Output**
>
> The output of specifying model components (variable or constraints)
> can be quite large for this problem because of the number of time
> periods. If you end a cell with an `@variable` or `@constraint`
> command, I *highly* recommend suppressing output by adding a
> semi-colon after the last command, or you might find that your
> notebook crashes.

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

The NY state legislature is considering enacting an annual
CO<sub>2</sub> limit, which for the utility would limit the emissions in
its footprint to 1.5 MtCO<sub>2</sub>/yr.

**In this problem**:

-   Reformulate your linear program from Problem 2 with any necessary
    changes to capture the CO<sub>2</sub> limit.
-   Implement the new optimization problem and find the optimal
    solution. How much should the utility build of each type of
    generating plant? What is different from your plan from Problem 1?
    Do these changes make sense?
-   What would the value to the utility be of allowing it to emit an
    additional 1000 tCO<sub>2</sub>/yr? An additional 5000?

## References

List any external references consulted, including classmates.